## Setup Only for Colab

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/hidden_mediators

In [ ]:
%ls

In [ ]:
from IPython.display import clear_output

In [ ]:
import time
!pip install -r requirements.txt
time.sleep(2)
clear_output()

In [ ]:
import time
# replace `develop` with `install` if you wont make library code changes
!python setup.py develop
time.sleep(2)
clear_output()
# Restart the session after running this

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

# Main Logic

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import pytest
from joblib import Parallel, delayed
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from proximalde.gen_data import gen_data_complex, gen_data_no_controls, gen_data_no_controls_discrete_m
from proximalde.proximal import proximal_direct_effect, ProximalDE, estimate_nuisances, \
    estimate_final, second_stage, _check_input, residualizeW, _gen_subsamples, RegularizedDualIVSolver
from proximalde.inference import NormalInferenceResults, pvalue
from proximalde.ivreg import Regularized2SLS, AdvIV
from proximalde.ivtests import weakiv_tests
from proximalde.tests.utilities import gen_iv_data
from sklearn.linear_model import RidgeCV, Ridge, LinearRegression, LassoCV
from sklearn.base import clone

```
a : strength of D -> M edge
b : strength of M -> Y edge
c : strength of D -> Y edge
d : strength of D -> Z edge
e : strength of M -> Z edge
f : strength of M -> X edge
g : strength of X -> Y edge
```

# Examining the Quality of Different Variants of the Dual Violation Test

In [ ]:
np.random.seed(12356)

def exp(it, variant, n, pz, px, e):
    np.random.seed(it)
    W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
    D = D.reshape(-1, 1)
    D = D - D.mean(axis=0)
    X = X - X.mean(axis=0)
    Z = Z - Z.mean(axis=0)

    if variant == 0:
        # ignore influence from estimation of gamma
        phi = np.zeros(X.shape)
        for train, test in KFold(2, shuffle=True, random_state=3).split(X, D):
            est = AdvIV(alphas=[1.0 * n**(0.3)]).fit(X[train], Z[train], D[train])
            gamma = est.coef_
            phi[test] = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))

        moments = np.mean(phi, axis=0)
        phi = phi - moments.reshape(1, -1)
        cov = (phi.T @ phi) / n
        return n * moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 1:
        # use Neyman orthogonal score, which involes estimating the projection
        # matrix P = E[XZ] E[XZ]^+. Which we estimate via a regularized SVD
        phi = np.zeros(X.shape)
        for train, test in KFold(3, shuffle=True, random_state=3).split(X, D):
            est = AdvIV(alphas=[1.0 * n**(0.3)]).fit(X[train], Z[train], D[train])
            gamma = est.coef_
            U, S, V = scipy.linalg.svd((X[train].T @ Z[train]) / len(train), full_matrices=False)
            P = U @ np.diag((S / (S + n**(-0.1)))) @ U.T
            Proj = np.eye(X.shape[1]) - P
            phi[test] = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))
            phi[test] = phi[test] @ Proj.T

        moments = np.mean(phi, axis=0)
        phi = phi - moments.reshape(1, -1)
        cov = (phi.T @ phi) / n
        return n * moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 2:
        # Use plug-in score and calculate the influence from gamma,
        # which ends up being the linear representation sqrt{n} (I - P) E_n[X (D-gamma Z)]
        phi = np.zeros(X.shape)
        for train, test in KFold(3, shuffle=True, random_state=3).split(X, D):
            est = AdvIV(alphas=[1.0 * n**(0.3)]).fit(X[train], Z[train], D[train])
            gamma = est.coef_
            phi[test] = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))

        moments = np.mean(phi, axis=0)

        U, S, V = scipy.linalg.svd((X.T @ Z) / n, full_matrices=False)
        P = U @ np.diag((S / (S + n**(-0.1)))) @ U.T
        Proj = np.eye(X.shape[1]) - P
        phi = phi @ Proj.T
        cov = (phi.T @ phi) / n
        return n * moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 3:
        # same as variant 1, but without cross-fitting; ignoring influence from gamma
        train, test = train_test_split(np.arange(X.shape[0]), test_size=.3)
        est = AdvIV(alphas=[1.0 * len(train)**(0.3)]).fit(X[train], Z[train], D[train])
        gamma = est.coef_
        phi = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))
        moments = np.mean(phi, axis=0)
        phi = phi - moments.reshape(1, -1)
        cov = (phi.T @ phi) / phi.shape[0]
        return phi.shape[0] * moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 4:
        # same as variant 2 but without cross-fitting. Now the influence
        # from gamma is sqrt{n//2} P E_{train}[X(D-gamma Z)]
        train, test = train_test_split(np.arange(X.shape[0]), test_size=.3)
        est = AdvIV(alphas=[1.0 * len(train)**(0.3)]).fit(X[train], Z[train], D[train])
        gamma = est.coef_
        U, S, V = scipy.linalg.svd((X[train].T @ Z[train]) / len(train), full_matrices=False)
        P = U @ np.diag(S / (S + 1 / n**(0.2))) @ U.T
        phi = X * (D - Z @ gamma.reshape(-1, 1))
        phi[train] = phi[train] @ P.T

        moments = np.mean(phi[test], axis=0)
        phi[test] = phi[test] - moments.reshape(1, -1)
        cov = (phi[test].T @ phi[test]) / phi[test].shape[0]**2
        cov += (phi[train].T @ phi[train]) / phi[train].shape[0]**2

        return moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 5:
        # Similar to variant 4, but using the regularized influence function
        # from the adviv, which couples the regularization used in the pseudo-inverse
        # with the one used to estimate gamma and not a different regularized SVD
        train, test = train_test_split(np.arange(X.shape[0]), test_size=.3, shuffle=True, random_state=it)
        est = AdvIV(alphas=[1.0 * len(train)**(0.3)]).fit(X[train], Z[train], D[train])
        gamma = est.coef_
        phi = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))

        moments = np.mean(phi, axis=0)
        phi = phi - moments.reshape(1, -1)
        cov = (phi.T @ phi) / phi.shape[0]**2
        der = X.T @ Z / X.shape[0]
        cov_gamma = est.inf_.T @ est.inf_ / (est.inf_.shape[0]**2)
        cov += der @ cov_gamma @ der.T

        return moments.T @ scipy.linalg.pinvh(cov) @ moments

In [ ]:
egrid = [0.0, 1.0]
variantgrid = [4]
ngrid = [50000, 500000]
pzpxgrid = [(1, 1), (5, 3), (100, 50)]
nexps = 1000

In [ ]:
res = {}
for e in egrid:
    res[e] = {}
    for variant in variantgrid:
        res[e][variant] = {}
        for n in ngrid:
            res[e][variant][n] = {}
            for pz, px in pzpxgrid:
                res[e][variant][n][(pz,px)] = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, variant, n, pz, px, e)
                                                                             for it in range(nexps))

In [ ]:
for e in egrid:
    print(f"e = {e}")
    for variant in variantgrid:
        print(f"Variant {variant}")
        for n in ngrid:
            for pz, px in pzpxgrid:
                result = np.array(res[e][variant][n][(pz,px)])
                print(f"({n}, {pz}, {px}): {np.mean(result > scipy.stats.chi2(df=px).ppf(.95))} vs 0.05, "
                    f"{np.mean(np.array(result), axis=0):.3f} vs {px}, "
                    f"{np.var(np.array(result), axis=0):.3f} vs {2*px}")

In [ ]:
for variant in variantgrid:
    print(f"Variant {variant}")
    plt.figure(figsize=(20, 5))
    it = 1
    for n in ngrid:
        for pz, px in pzpxgrid:
            result = np.array(res[egrid[-1]][variant][n][(pz,px)])
            result_violated = {}
            for e in egrid[:-1]:
                result_violated[e] = np.array(res[e][variant][n][(pz,px)])
            plt.subplot(len(ngrid), len(pzpxgrid), it)
            plt.title(f"({n}, {pz}, {px}): {np.mean(result > scipy.stats.chi2(df=px).ppf(.95))} vs 0.05, "
                      f"{np.mean(np.array(result), axis=0):.3f} vs {px}, "
                      f"{np.var(np.array(result), axis=0):.3f} vs {2*px}\n"
                      + ", ".join(f"Power[{e}]: {np.mean(result_violated[e] > scipy.stats.chi2(df=px).ppf(.95))} vs 1.0" for e in egrid[:-1]))
            plt.hist(result)
            plt.axvline(scipy.stats.chi2(df=px).ppf(.95), color='r')
            for e in egrid[:-1]:
                plt.hist(result_violated[e], alpha=0.4)
            it += 1
    plt.tight_layout()
    plt.show()

In [ ]:
from statsmodels.graphics.gofplots import qqplot
for variant in variantgrid:
    print(f"Variant {variant}")
    plt.figure(figsize=(20, 7))
    it = 1
    for n in ngrid:
        for pz, px in pzpxgrid:
            result = np.array(res[egrid[-1]][variant][n][(pz,px)])
            ax = plt.subplot(len(ngrid), len(pzpxgrid), it)
            plt.title(f"({n}, {pz}, {px}): {np.mean(result > scipy.stats.chi2(df=px).ppf(.95))} vs 0.05, "
                    f"{np.mean(np.array(result), axis=0):.3f} vs {px}, "
                    f"{np.var(np.array(result), axis=0):.3f} vs {2*px}")
            qqplot(result, dist=scipy.stats.chi2(df=px), line='45', ax=ax)
            it += 1
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
n = 50000
e = 1.0
pz, px = 60, 5
def exp(it):
    np.random.seed(it)
    W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
    est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=it, verbose=0)
    est.fit(None, D, Z, X, Y)
    return est.dual_violation_test()[0]

res = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it) for it in range(1000))

In [ ]:
plt.title(f"{np.mean(res > scipy.stats.chi2(df=px).ppf(.95))} vs 0.05, {np.mean(np.array(res), axis=0):.3f} vs {px}, {np.var(np.array(res), axis=0):.3f} vs {2*px}")
plt.hist(res)
plt.axvline(scipy.stats.chi2(df=px).ppf(.95))
# plt.hist(np.random.chisquare(df=px, size=(100,)), alpha=.3,)
plt.show()

In [ ]:
from statsmodels.graphics.gofplots import qqplot
qqplot(np.array(res), dist=scipy.stats.chi2(df=px), line='45')
plt.show()

# Multi-Dimensional Mediator

In [ ]:
np.random.seed(12356)

def exp(it, variant, n, pz, px, E, F, pm):
    np.random.seed(it)
    W, D, _, Z, X, Y = gen_data_no_controls_discrete_m(n, pw, pz, px, a, b, c, d, E, F, g, pm=pm)
    D = D.reshape(-1, 1)
    D = D - D.mean(axis=0)
    X = X - X.mean(axis=0)
    Z = Z - Z.mean(axis=0)

    if variant == 0:
        # ignore influence from estimation of gamma
        phi = np.zeros(X.shape)
        for train, test in KFold(2, shuffle=True, random_state=3).split(X, D):
            est = AdvIV(alphas=[1.0 * n**(0.3)]).fit(X[train], Z[train], D[train])
            gamma = est.coef_
            phi[test] = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))

        moments = np.mean(phi, axis=0)
        phi = phi - moments.reshape(1, -1)
        cov = (phi.T @ phi) / n
        return n * moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 1:
        # use Neyman orthogonal score, which involes estimating the projection
        # matrix P = E[XZ] E[XZ]^+. Which we estimate via a regularized SVD
        phi = np.zeros(X.shape)
        for train, test in KFold(3, shuffle=True, random_state=3).split(X, D):
            est = AdvIV(alphas=[1.0 * n**(0.3)]).fit(X[train], Z[train], D[train])
            gamma = est.coef_
            U, S, V = scipy.linalg.svd((X[train].T @ Z[train]) / len(train), full_matrices=False)
            P = U @ np.diag((S / (S + n**(-0.1)))) @ U.T
            Proj = np.eye(X.shape[1]) - P
            phi[test] = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))
            phi[test] = phi[test] @ Proj.T

        moments = np.mean(phi, axis=0)
        phi = phi - moments.reshape(1, -1)
        cov = (phi.T @ phi) / n
        return n * moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 2:
        # Use plug-in score and calculate the influence from gamma,
        # which ends up being the linear representation sqrt{n} (I - P) E_n[X (D-gamma Z)]
        phi = np.zeros(X.shape)
        for train, test in KFold(3, shuffle=True, random_state=3).split(X, D):
            est = AdvIV(alphas=[1.0 * n**(0.3)]).fit(X[train], Z[train], D[train])
            gamma = est.coef_
            phi[test] = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))

        moments = np.mean(phi, axis=0)

        U, S, V = scipy.linalg.svd((X.T @ Z) / n, full_matrices=False)
        P = U @ np.diag((S / (S + n**(-0.1)))) @ U.T
        Proj = np.eye(X.shape[1]) - P
        phi = phi @ Proj.T
        cov = (phi.T @ phi) / n
        return n * moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 3:
        # same as variant 1, but without cross-fitting; ignoring influence from gamma
        train, test = train_test_split(np.arange(X.shape[0]), test_size=.3)
        est = AdvIV(alphas=[1.0 * len(train)**(0.3)]).fit(X[train], Z[train], D[train])
        gamma = est.coef_
        phi = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))
        moments = np.mean(phi, axis=0)
        phi = phi - moments.reshape(1, -1)
        cov = (phi.T @ phi) / phi.shape[0]
        return phi.shape[0] * moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 4:
        # same as variant 2 but without cross-fitting. Now the influence
        # from gamma is sqrt{n//2} P E_{train}[X(D-gamma Z)]
        train, test = train_test_split(np.arange(X.shape[0]), test_size=.3)
        est = AdvIV(alphas=[1.0 * len(train)**(0.3)]).fit(X[train], Z[train], D[train])
        gamma = est.coef_
        U, S, V = scipy.linalg.svd((X[train].T @ Z[train]) / len(train), full_matrices=False)
        P = U @ np.diag(S / (S + 1 / n**(0.2))) @ U.T
        phi = X * (D - Z @ gamma.reshape(-1, 1))
        phi[train] = phi[train] @ P.T

        moments = np.mean(phi[test], axis=0)
        phi[test] = phi[test] - moments.reshape(1, -1)
        cov = (phi[test].T @ phi[test]) / phi[test].shape[0]**2
        cov += (phi[train].T @ phi[train]) / phi[train].shape[0]**2

        return moments.T @ np.linalg.pinv(cov) @ moments
    if variant == 5:
        # Similar to variant 4, but using the regularized influence function
        # from the adviv, which couples the regularization used in the pseudo-inverse
        # with the one used to estimate gamma and not a different regularized SVD
        train, test = train_test_split(np.arange(X.shape[0]), test_size=.3, shuffle=True, random_state=it)
        est = AdvIV(alphas=[1.0 * len(train)**(0.3)]).fit(X[train], Z[train], D[train])
        gamma = est.coef_
        phi = X[test] * (D[test] - Z[test] @ gamma.reshape(-1, 1))

        moments = np.mean(phi, axis=0)
        phi = phi - moments.reshape(1, -1)
        cov = (phi.T @ phi) / phi.shape[0]**2
        der = X.T @ Z / X.shape[0]
        cov_gamma = est.inf_.T @ est.inf_ / (est.inf_.shape[0]**2)
        cov += der @ cov_gamma @ der.T

        return moments.T @ scipy.linalg.pinvh(cov) @ moments

In [ ]:
pm = 7
pzpxgrid = [(20, 10), (100, 50)]
ngrid = [50000, 500000]
egrid = [0.0, 1.0]
variantgrid = [4]
nexps = 1000

In [ ]:
res = {}
for pz, px in pzpxgrid:
    full_rank = False
    while not full_rank:
        E = np.random.normal(0, 2, (pm, pz))
        F = np.random.normal(0, 2, (pm, px))
        if (np.linalg.matrix_rank(E, tol=0.5) == pm) and (np.linalg.matrix_rank(F, tol=0.5) == pm):
            full_rank = True
    res[(pz, px)] = {}
    for e in egrid:
        res[(pz, px)][e] = {}
        for variant in variantgrid:
            res[(pz, px)][e][variant] = {}
            for n in ngrid:
                res[(pz, px)][e][variant][n] = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, variant, n, pz, px, e*E, f*F, pm) for it in range(nexps))

In [ ]:
for pz, px in pzpxgrid:
    for e in egrid:
        print(f"e = {e}")
        for variant in variantgrid:
            print(f"Variant {variant}")
            for n in ngrid:

                result = np.array(res[(pz, px)][e][variant][n])
                print(f"({n}, {pz}, {px}): {np.mean(result > scipy.stats.chi2(df=px).ppf(.95))} vs 0.05, "
                    f"{np.mean(np.array(result), axis=0):.3f} vs {px}, "
                    f"{np.var(np.array(result), axis=0):.3f} vs {2*px}")

In [ ]:
for variant in variantgrid:
    print(f"Variant {variant}")
    plt.figure(figsize=(20, 5))
    it = 1
    for n in ngrid:
        for pz, px in pzpxgrid:
            result = np.array(res[(pz,px)][egrid[-1]][variant][n])
            result_violated = {}
            for e in egrid[:-1]:
                result_violated[e] = np.array(res[(pz,px)][e][variant][n])
            plt.subplot(len(ngrid), len(pzpxgrid), it)
            plt.title(f"({n}, {pz}, {px}): {np.mean(result > scipy.stats.chi2(df=px).ppf(.95))} vs 0.05, "
                      f"{np.mean(np.array(result), axis=0):.3f} vs {px}, "
                      f"{np.var(np.array(result), axis=0):.3f} vs {2*px}\n"
                      + ", ".join(f"Power[{e}]: {np.mean(result_violated[e] > scipy.stats.chi2(df=px).ppf(.95))} vs 1.0" for e in egrid[:-1]))
            plt.hist(result)
            plt.axvline(scipy.stats.chi2(df=px).ppf(.95), color='r')
            for e in egrid[:-1]:
                plt.hist(result_violated[e], alpha=0.4)
            it += 1
    plt.tight_layout()
    plt.show()

In [ ]:
np.random.seed(it)
W, D, _, Z, X, Y = gen_data_no_controls_discrete_m(n, pw, pz, px, a, b, c, d, E, F, g, sm=sm, pm=pm)
est = ProximalDE().fit(None, D, Z, X, Y)


# Examining that Tests Capture the Failure Modes they Intend to Capture

In [ ]:
# W, D, _, Z, X, Y = gen_data_complex(n, pw, pz, px, a, b, c, d, e, f, g)

In [ ]:
def true_params(pw, pz, px, a, b, c, d, e, f, g, sm, sz=1.0, sd=0.5):
    if pz == 1 and px == 1:
        true_Zsq = (e * a + d)**2 * sd**2 + e**2 * sm**2 + sz**2
        true_Msq = sm**2 + a**2 * sd**2
        true_XZ = f * (e * true_Msq + d * a * sd**2)
        true_DZ = (a * e + d) * sd**2
        true_DX = a * f * sd**2
        # D*X / X*Z
        if true_XZ == 0:
            true_gamma = np.inf
        else:
            true_gamma = true_DX / true_XZ
        # D^2 - gamma D * Z
        true_strength = sd**2 - true_gamma * true_DZ
        return true_gamma, true_strength
    else:
        raise AttributeError("Not available")

In [ ]:
def exp_summary(it, n, pw, pz, px, a, b, c, d, e, f, g, sm):
    np.random.seed(it)
    W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
    est = ProximalDE(dual_type='Z', cv=3, semi=True, ivreg_type='adv',
                     multitask=False, n_jobs=1, random_state=3, verbose=0)
    est.fit(None, D, Z, X, Y)
    lb, ub = est.robust_conf_int(lb=-2, ub=2)
    weakiv_stat, _, _, _, pi, var_pi = est.weakiv_test(return_pi_and_var=True)
    eigs, eig_crit = est.covariance_rank_test(calculate_critical=True)
    maxeig = eigs[0]
    return est.stderr_, est.idstrength_, est.primal_violation_, est.dual_violation_, est.point_, \
        lb, ub, weakiv_stat, maxeig, pi, var_pi, *est.gamma_.flatten(), *est.ivreg_gamma_.stderr_.flatten(), est.idstrength_std_, eig_crit

def run_summary(n, pw, pz, px, a, b, c, d, e, f, g, sm):
    res = np.array(Parallel(n_jobs=-1, verbose=3)(delayed(exp_summary)(it, n, pw, pz, px, a, b, c, d, e, f, g, sm)
                                                  for it in range(100)))
    print(f"Mean estimate: {np.mean(res[:, 4]):.3f}")
    print(f"Bias: {np.mean(res[:, 4] - c):.3f}")
    print(f"RMSE: {np.sqrt(np.mean((res[:, 4] - c)**2)):.3f}")
    cov = np.mean((res[:, 4] + 1.96 * res[:, 0] >= c) & (res[:, 4] - 1.96 * res[:, 0] <= c))
    print(f"Coverage: {cov:.3f}")
    rcov = np.mean((res[:, 5] <= c) & (res[:, 6] >= c))
    print(f"ID-Robust Coverage: {rcov:.3f}")
    plt.figure(figsize=(10, 10))
    plt.subplot(3, 2, 1)
    plt.title(f"stderr: mean={np.mean(res[:, 0]):.3f}, %>1.0={np.mean(res[:, 0] > 2.0):.3f}")
    plt.hist(res[:, 0])
    plt.subplot(3, 2, 2)
    crit = np.round(scipy.stats.foldnorm(c=10).ppf(.95), 2)
    plt.title(f"idstrength: mean={np.mean(res[:, 1] / res[:, 11 + 2*pz]):.3f}, %<{crit}={np.mean(res[:, 1] / res[:, 11 + 2*pz] < crit):.3f}")
    plt.hist(res[:, 1] / res[:, 11 + 2*pz])
    plt.subplot(3, 2, 3)
    crit = np.round(scipy.stats.chi2(df=pz + 1).ppf(.95), 2)
    plt.title(f"primal_violation: mean={np.mean(res[:, 2]):.3f}, %>{crit}={np.mean(res[:, 2] > crit):.3f}")
    plt.hist(res[:, 2])
    plt.subplot(3, 2, 4)
    crit = np.round(scipy.stats.chi2(df=px).ppf(.95), 2)
    plt.title(f"dual_violation: mean={np.mean(res[:, 3]):.3f}, %>{crit}={np.mean(res[:, 3] > crit):.3f}")
    plt.hist(res[:, 3])
    plt.subplot(3, 2, 5)
    plt.title(f"weakiv_test: mean={np.mean(res[:, 7]):.3f}, %<23={np.mean(res[:, 7] < 23):.3f}")
    plt.hist(res[:, 7])
    plt.subplot(3, 2, 6)
    eig_stat = res[:, 8] / res[:, 11+2*pz+1]
    plt.title(f"rank_test: mean={np.mean(eig_stat):.3f}, %<1={np.mean(eig_stat < 1):.3f}")
    plt.hist(eig_stat)
    plt.tight_layout()
    plt.show()

    pi = res[:, 9]
    var_pi = n * res[:, 10]
    print(f"mean(pi) = {np.mean(pi):.3f}")
    print(f"True n * var(pi)={n * np.var(pi):.3f}")
    print(f"n * hatvar(pi): mean={np.mean(var_pi):.3f}, median={np.median(var_pi):.3f}, "
          f"(5%, 95%)=({np.percentile(var_pi, 5):.3f}, {np.percentile(var_pi, 95):.3f})"
          f"(1%, 99%)=({np.percentile(var_pi, 1):.3f}, {np.percentile(var_pi, 99):.3f})")

    gamma = res[:, 11:11+pz]
    var_gamma = n * (res[:, 11+pz:11+2*pz])**2
    print("mean(gamma) = ", np.round(np.mean(gamma, axis=0), 3))
    print("True n * var(gamma)=", np.round(n * np.var(gamma, axis=0), 3))
    print("n * hatvar(gamma): mean=", np.round(np.mean(var_gamma, axis=0), 3), "median=", np.round(np.median(var_gamma, axis=0), 3),
          "(5%, 95%)=(", np.round(np.percentile(var_gamma, 5, axis=0), 3), np.round(np.percentile(var_gamma, 95, axis=0),3), ")")

    idstrength = res[:, 1]
    var_idstrength = n * (res[:, 11+2*pz])**2
    print("mean(str) = ", np.round(np.mean(idstrength, axis=0), 3))
    print("True n * var(str)=", np.round(n * np.var(idstrength, axis=0), 3))
    print("n * hatvar(str): mean=", np.round(np.mean(var_idstrength, axis=0), 3), "median=", np.round(np.median(var_idstrength, axis=0), 3),
          "(5%, 95%)=(", np.round(np.percentile(var_idstrength, 5, axis=0), 3), np.round(np.percentile(var_idstrength, 95, axis=0),3), ")")

    if pz == 1 and px == 1:
        true_gamma, _ = true_params(pw, pz, px, a, b, c, d, e, f, g, sm)
        W, D, _, Z, X, Y = gen_data_no_controls(100000, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
        D = D.reshape(-1, 1)
        D = D - D.mean(axis=0)
        X = X - X.mean(axis=0)
        Z = Z - Z.mean(axis=0)
        print(f"true(pi)={LinearRegression(fit_intercept=False).fit(D - true_gamma * Z, D.flatten()).coef_[0]:.3f}")

In [ ]:
def test_pi_and_var_pi():
    np.random.seed(123)
    n = 10000
    pw = 1
    pz, px = 3, 3
    n_splits = 3
    # Indirect effect is a*b, direct effect is c
    a, b, c = 1.0, 1.0, .5
    # D has direct relationship to Z, Z has direct relationship to M,
    # X has direct relationship to M, X has direct relationship to Y
    d, e, f, g = 1.0, 1.0, 1.0, 1.0
    sm = 2.0
    res = np.array(Parallel(n_jobs=-1, verbose=3)(delayed(exp_summary)(it, n, pw, pz, px, a, b, c, d, e, f, g, sm)
                                                  for it in range(100)))

    pi = res[:, 9]
    var_pi = n * res[:, 10]
    print(np.mean(pi))
    print(n * np.var(pi), np.mean(var_pi))
    print(n * np.var(pi), np.percentile(var_pi, 1))
    print(n * np.var(pi), np.percentile(var_pi, 99))
    assert np.allclose(n * np.var(pi), np.mean(var_pi), atol=5e-3)
    assert np.allclose(n * np.var(pi), np.percentile(var_pi, 1), atol=5e-3)
    assert np.allclose(n * np.var(pi), np.percentile(var_pi, 99), atol=5e-3)

In [ ]:
# test_pi_and_var_pi()

In [ ]:
# exp_summary(0, n, pw, pz, px, a, b, c, d, e, f, g, sm)

In [ ]:
n = 10000
pw = 1
pz, px = 100, 50
n_splits = 3
np.random.seed(123)

## Failure mode 1

D has direct relationship to Z, but Z has no direct relationship from M. So Z is not a good proxy treatment. However, X is a good proxy outcome and this can allow us to detect the failure mode. Here the fact that Z is correlated with D, makes the dual assumption not be violated. However, the solution basically leads to an orthogonal instrument that perfectly predicts treatment and hence the identification strength assumption is violated.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has direct relationship to Z, Z has no relationship to M,
# X has direct relationship to M, X has no direct relationship to Y
d, e, f, g = 1.0, 0.0, 1.0, 0.0
sm = 2.0
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
est = ProximalDE(dual_type='Z', cv=3, semi=True, ivreg_type='adv',
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(None, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])
# we see that even though both primal and dual have feasible solutions
# the identification is very weak, since Z is only driven by D and
# hence D can almost perfectly predict D. So the id_stregth test failed
# and caught this failure mode. Also confidence intervals are quite large
# so we are not artificially confident about a wrong result.

In [ ]:
est.summary()

In [ ]:
run_summary(n, pw, pz, px, a, b, c, d, e, f, g, sm)

## Failure mode 2

D has no direct relationship to Z, but Z has no direct relationship from M. So Z is not a good proxy treatment. However, X is a good proxy outcome and this can allow us to detect the failure mode. Her Z is un-correlated with D, which makes the dual assumption violated.

Because the dual doesn't have a solution, the weakIV test is invalid, as the covariance calculation depends on the validity of the dual moment.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has no direct relationship to Z, Z has no direct relationship to M,
# X has direct relationship to M, X has no direct relationship to Y
d, e, f, g = 0.0, 0.0, 1.0, 0.0
sm = 2.0
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(None, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])
# we see that in this case even though the idstrength test passed
# the dual violation test did not pass. In this case the dual problem
# does not admit a solution and this was detected by the dual_violation
# statistic which was very high

In [ ]:
run_summary(n, pw, pz, px, a, b, c, d, e, f, g, sm)

## Failure Mode 3


D is a good proxy outcome, but X is not and is un-related to M. This makes the existence of a solution to the primal IV assumption violated.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has no direct relationship to Z, Z has direct relationship to M,
# X has no direct relationship to M, X has no direct relationship to Y
d, e, f, g = 0.0, 1.0, 0.0, 0.0
sm = 2.0
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(None, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])
# we see that in this case even though the idstrength test passed
# and the dual violation test passed, the primal violation almost did
# not pass. This test should be catching this failure.

In [ ]:
run_summary(n, pw, pz, px, a, b, c, d, e, f, g, sm)

## Failure mode 4


Z and X are unrelated to all other variables!

The rank test can catch this failure mode.

Sometimes the dual violation will catch this failure mode.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has no direct relationship to Z, Z has no direct relationship to M,
# X has no direct relationship to M, X has no direct relationship to Y
d, e, f, g = 0.0, 0.0, 0.0, 0.0
sm = 2.0
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(None, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])

In [ ]:
run_summary(n, pw, pz, px, a, b, c, d, e, f, g, sm)

## Failure Mode 5

Almost the same as mode 4, but only X is related to Y

The rank test catches this failure mode.

Sometimes the primal/dual violation, catch this failure mode.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has no direct relationship to Z, Z has no direct relationship to M,
# X has no direct relationship to M, X has direct relationship to Y
d, e, f, g = 0.0, 0.0, 0.0, 5.0
sm = 2.0
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(None, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])

In [ ]:
run_summary(n, pw, pz, px, a, b, c, d, e, f, g, sm)

## Failure Mode 6

X, Z are good proxies, but the mediator is super correlated with the treatment, which leads to lack of identification of the direct effect.

The weakIV and idstrength tests will catch this case.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has direct relationship to Z, Z has direct relationship to M,
# X has direct relationship to M, X has direct relationship to Y
d, e, f, g = 1.0, 1.0, 1.0, 1.0
sm = 0.05
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(None, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])

In [ ]:
run_summary(n, pw, pz, px, a, b, c, d, e, f, g, sm)

## No Failure Mode

In [ ]:
np.random.seed(1)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has direct relationship to Z, Z has direct relationship to M,
# X has direct relationship to M, X has direct relationship to Y
d, e, f, g = 1.0, 1.0, 1.0, 1.0
sm = 2.0
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, sm=sm)
est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(None, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])

In [ ]:
run_summary(n, pw, pz, px, a, b, c, d, e, f, g, sm)

# OLD

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# Z has no relationship to M, but X has relationship to M
# D also has direct relationship to Z, X doesn't have direct relationship to Y
d, e, f, g = 1.0, 0.0, 1.0, 0.0
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g)
est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(W, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])
# we see that even though both primal and dual have feasible solutions
# the identification is very weak, since Z is only driven by D and
# hence D can almost perfectly predict D. So the id_stregth test failed
# and caught this failure mode.

In [ ]:
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# Z has no relationship to M, but X has relationship to M
# D also has direct relationship to Z, X doesn't have direct relationship to Y
d, e, f, g = 1.0, 0.0, 1.0, 0.0
W, D, _, Z, X, Y = gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g)
est = ProximalDE(dual_type='Z', cv=3, semi=True,
                 multitask=False, n_jobs=-1, random_state=3, verbose=0)
est.fit(W, D, Z, X, Y)
display(est.summary().tables[0])
display(est.summary().tables[1])
display(est.summary().tables[2])

In [ ]:
np.mean(est.Dres_ * est.Xres_)

In [ ]:
np.mean(est.Zres_ * est.Xres_)

In [ ]:
est.summary()

In [ ]:
diag = est.run_diagnostics()

In [ ]:
inds = est.influential_set(alpha=0.05)

In [ ]:
from sklearn.base import clone
est2 = clone(est)
est2.fit(np.delete(W, inds, axis=0), np.delete(D, inds, axis=0),
         np.delete(Z, inds, axis=0), np.delete(X, inds, axis=0),
         np.delete(Y, inds, axis=0))

In [ ]:
est2.summary(alpha=0.05)

#### Verifying we get the same as ivreg on the dataset in the documentation of ivreg

In [ ]:
from proximalde.tests.utilities import gen_kmenta_data
Z, X, Y, labels, controls = gen_kmenta_data()

In [ ]:
from proximalde.ivtests import weakiv_tests
weakiv_tests(Z, X, Y, controls=controls)

In [ ]:
from proximalde.diagnostics import IVDiagnostics
diag = IVDiagnostics(add_constant=True).fit(Z, X, Y)

In [ ]:
diag.influence_plot(labels=labels)
plt.show()